In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load simulation data from CSV
df = pd.read_csv("scrabble_strategy_data.csv")

strategy_map = {
    "Plato": "GREEDY",
    "Pythagoras": "GREEDY",
    "Aristotle": "BALANCED",
    "Socrates": "BALANCED"
}
df['Strategy'] = df['Player'].map(strategy_map)

# Calculate earned score per move
df['Earned'] = df.groupby(['Game', 'Player'])['Score'].diff().fillna(0)

# Draw Plot line chart for each game
unique_games = df['Game'].unique()
for game_id in unique_games:
    game_data = df[df['Game'] == game_id].copy()
    game_data['Label'] = game_data['Player'] + " (" + game_data['Strategy'] + ")"

    # Determine winner for this game
    last_turn = game_data['Turn'].max()
    final_scores = game_data[game_data['Turn'] == last_turn]
    winner_row = final_scores.loc[final_scores['Score'].idxmax()]
    winner_strategy = winner_row['Strategy']
    winner_name = winner_row['Player']

    # Plot
    fig = px.line(
        game_data,
        x='Turn',
        y='Earned',
        color='Label',
        title=f"Game {game_id} - Earned Score per Turn (Winner: {winner_name}, Strategy: {winner_strategy})",
        markers=True
    )
    fig.show()

# Draw bar chart for total earned score per strategy bar chart
total_earned = df.groupby('Strategy')['Earned'].sum().reset_index()

fig = px.bar(
    total_earned,
    x='Strategy',
    y='Earned',
    color='Strategy',
    color_discrete_map={'GREEDY': 'red', 'BALANCED': 'blue'},
    title="Total Earned Score by Strategy Across All Games"
)
fig.update_layout(xaxis_title='Strategy', yaxis_title='Total Earned Score')
fig.show()
